# エージェントを構築する
言語モデル自体はアクションを実行できず、テキストを出力するだけです。LangChainの大きなユースケースはエージェントの作成です。[エージェント](https://python.langchain.com/docs/concepts/agents/)は、[LLM](https://python.langchain.com/docs/concepts/chat_models/)を推論エンジンとして使用し、実行するアクションと必要な入力を決定するシステムです。アクションを実行した後、その結果をLLMにフィードバックして、さらにアクションが必要かどうか、または終了してもよいかどうかを判断します。これはしばしば[ツール呼び出し](https://python.langchain.com/docs/concepts/tool_calling/)を介して達成されます。

このチュートリアルでは、検索エンジンと対話できるエージェントを構築します。このエージェントに質問をし、検索ツールを呼び出す様子を見て、それと会話することができます。

[Build an Agent \| 🦜️🔗 LangChain](https://python.langchain.com/docs/tutorials/agents/)

## エンドツーエンドのエージェント
以下のコードスニペットは、使用するツールを決定するためにLLMを使用する完全に機能するエージェントを表しています。これは汎用の検索ツールを備えています。会話のメモリを持っており、マルチターンのチャットボットとして使用できます。

ガイドの残りの部分では、個々のコンポーネントと各部分の役割について説明しますが、すぐにコードを取得して始めたい場合は、自由に使用してください！

In [0]:
%pip install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite langchain-openai mlflow
dbutils.library.restartPython()

In [0]:
import mlflow

# MLflow Tracingの有効化
mlflow.langchain.autolog()

In [0]:
import os

os.environ["OPENAI_API_KEY"] = dbutils.secrets.get("demo-token-takaaki.yayoi", "openai_api_key")
# TavilyのAPIキー
os.environ["TAVILY_API_KEY"] = "tvly-..."

## エージェントの作成

In [0]:
# 必要な機能をインポート
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_core.pydantic_v1 import BaseModel

# メモリー
memory = MemorySaver()
# 言語モデル
model = ChatOpenAI(model="gpt-4o-mini")
# 検索ツール
search = TavilySearchResults(max_results=2)
# ツールセット
tools = [search]
# エージェントの作成
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [0]:
from IPython.display import Image, display

display(Image(agent_executor.get_graph().draw_mermaid_png()))

## エージェントの実行

エージェントが複数のステップを実行する場合、時間がかかることがあります。中間の進捗状況を表示するために、発生したメッセージをストリーミングバックすることができます。

In [0]:
# エージェントを使います
config = {"configurable": {"thread_id": "abc123"}}
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="こんにちは、Takaです！私は東京に住んでいます。")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [0]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="私が住んでいる場所の天気はどうですか？")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

## トークンのストリーミング

メッセージをストリーミングバックするだけでなく、トークンをストリーミングバックすることも有用です。これを行うには、`stream_mode="messages"`を指定します。

In [0]:
for step, metadata in agent_executor.stream(
    {"messages": [HumanMessage(content="東京の天気は？")]},
    config,
    stream_mode="messages",
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

## まとめ

これで終了です！このクイックスタートでは、シンプルなエージェントの作成方法を紹介しました。その後、応答をストリーミングバックする方法を示しました - 中間ステップだけでなく、トークンも含めて！さらに、会話を続けるためのメモリも追加しました。エージェントは学ぶことがたくさんある複雑なトピックです！

エージェントに関する詳細情報は、[LangGraph](https://python.langchain.com/docs/concepts/architecture/#langgraph)のドキュメントを参照してください。独自の概念、チュートリアル、およびハウツーガイドが含まれています。